# Current Status

* Infrastructure for calculating probabilities is defined
* Only one scenario is defined
* needs better UI

In [7]:
unit_stats = {
    'level1': {
        'attack': 2,
        'defense': 4
    },
    'level2': {
        'attack': 3,
        'defense': 5
    }
}

action_stats = {
    'level1': {
        'hasty': -1,
        'deliberate': 1
    },
    'level2': {
        'hasty': 0,
        'deliberate': 2
    }
}

adjacent_unit_defense_bonus = 1

In [8]:
def get_required_roll_to_win(num_lvl1_attackers, num_lvl1_defenders, num_lvl2_attackers, num_lvl2_defenders, action_modifier, num_adjacent_defenders):
    attack_modifier = (num_lvl1_attackers * unit_stats['level1']['attack']) + (num_lvl2_attackers * unit_stats['level2']['attack']) + action_modifier
    defense_modifier = (num_lvl1_defenders * unit_stats['level1']['defense']) + (num_lvl2_defenders * unit_stats['level2']['defense']) + (num_adjacent_defenders * adjacent_unit_defense_bonus)
    return defense_modifier + 1 - attack_modifier

In [9]:
from dataclasses import dataclass

@dataclass
class UnitGroup:
    lvl1: int
    lvl2: int

@dataclass
class Scenario:
    name: str
    attackers: UnitGroup
    defenders: UnitGroup
    action_modifier: int
    adjacent_defenders: int


def get_scenario_roll_to_win(scenario):
    return get_required_roll_to_win(scenario.attackers.lvl1, scenario.defenders.lvl1, scenario.attackers.lvl2, scenario.defenders.lvl2, scenario.action_modifier, scenario.adjacent_defenders)


In [10]:
from scipy import stats
from scipy.stats import randint
import numpy as np
import itertools

def dice_distribution(num_dice, min_roll, max_roll):
    single_dice_possible_rolls = np.arange(min_roll, max_roll + 1)
    roll_range = max_roll + 1 - min_roll
    single_dice_probabilities = [1 / roll_range] * roll_range
    single_dice_zipped = zip(single_dice_possible_rolls, single_dice_probabilities)
    
    # all possible combinations of dice rolls
    cartesian_product = list(itertools.product(single_dice_zipped, repeat = num_dice))

    possible_results = {}
    for possible_combination in cartesian_product:
        total = 0
        probability = 1
        for (roll_result, roll_probability) in possible_combination:
            total += roll_result
            probability *= roll_probability
            
        if total not in possible_results:
            possible_results[total] = 0
        possible_results[total] += probability
        
    all_dice_possible_rolls = []
    all_dice_probabilities = []
    for possible_roll, probability in possible_results.items():
        all_dice_possible_rolls.append(possible_roll)
        all_dice_probabilities.append(probability)
        
    return stats.rv_discrete(values=(all_dice_possible_rolls, all_dice_probabilities))

distro_2d6 = dice_distribution(2, 1, 6)

single_dice = {
    "d4": (1, 5),
    "d6": (1, 7)
}


def probability_of_roll_geq_than_value(distro, value):
    return distro.sf(value - 1) # .sf() is survival function; returns chance of being > value, not >= value

distributions = {
    "1d4": dice_distribution(1, 1, 4),
    "2d4": dice_distribution(2, 1, 4),
    "1d6": dice_distribution(1, 1, 6),
    "2d6": dice_distribution(2, 1, 6)
}

In [14]:
scenarios = [
    Scenario("level 1 unit vs level 1 unit, hasty lvl1",
             UnitGroup(1, 0),
             UnitGroup(1, 0),
             action_stats["level1"]["hasty"],
             0
            )
]

diceroll = distributions["1d4"]
chance_to_win = probability_of_roll_geq_than_value(diceroll, get_scenario_roll_to_win(scenarios[0]))
print(chance_to_win)

0.25
